In [1]:
! pip install -Uqq pip llama-index llama-index-vector-stores-chroma llama-index-utils-workflow llama-index-llms-huggingface-api pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 71.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.7/808.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/

In [2]:
!git config --global credential.helper store

In [3]:
from huggingface_hub import login

login()

In [4]:
##@ basic workflow creation in llamaindex

from llama_index.core.workflow import StartEvent, StopEvent, Workflow, step

class MyWorkflow(Workflow):
    @step
    async def my_step(self, ev: StartEvent) -> StopEvent:
        return StopEvent(result="Namaste 🙏!")

w= MyWorkflow(timeout=10, verbose = False)
result = await w.run()

result

'Namaste 🙏!'

In [19]:
from llama_index.core.workflow import Event, StartEvent, StopEvent, Workflow, step
import asyncio

# Define custom event classes for each step
class GreetEvent(Event):
    message: str

class ServeWaterEvent(Event):
    message: str

class TakeOrderEvent(Event):
    message: str

class ServeFoodEvent(Event):
    message: str

class ProvideBillEvent(Event):
    message: str

class PaymentEvent(Event):
    message: str

class ConfirmPaymentEvent(Event):
    message: str

# Define the workflow class with logging
class MultiStepWorkflow(Workflow):
    @step
    async def greet(self, ev: StartEvent) -> GreetEvent:
        message = "Waiter: Greet 🙏 the guest"
        print(message)  # Log the step
        return GreetEvent(message=message)

    @step
    async def serve_water(self, ev: GreetEvent) -> ServeWaterEvent:
        message = "Waiter: Provide Guest with water 🥤"
        print(message)  # Log the step
        return ServeWaterEvent(message=message)

    @step
    async def take_order(self, ev: ServeWaterEvent) -> TakeOrderEvent:
        message = "Waiter: Provide guest the menu and confirm order"
        print(message)  # Log the step
        return TakeOrderEvent(message=message)

    @step
    async def serve_food(self, ev: TakeOrderEvent) -> ServeFoodEvent:
        message = "Waiter: Serve the guest 🍕"
        print(message)  # Log the step
        return ServeFoodEvent(message=message)

    @step
    async def provide_bill(self, ev: ServeFoodEvent) -> ProvideBillEvent:
        message = "Waiter: Provide guest his/her bill"
        print(message)  # Log the step
        return ProvideBillEvent(message=message)

    @step
    async def payment(self, ev: ProvideBillEvent) -> PaymentEvent:
        message = "Guest: Clear his/her bill 💵"
        print(message)  # Log the step
        return PaymentEvent(message=message)

    @step
    async def confirm_payment(self, ev: PaymentEvent) -> ConfirmPaymentEvent:
        message = "Waiter: Confirms the payment ✅"
        print(message)  # Log the step
        return ConfirmPaymentEvent(message=message)

    @step
    async def finish(self, ev: ConfirmPaymentEvent) -> StopEvent:
        message = f"Finished txn:: {ev.message}"
        print(message)  # Log the final step
        return StopEvent(result=message)

# Define the main async function
async def main():
    w = MultiStepWorkflow(timeout=10, verbose=False)
    result = await w.run()

# Execute the workflow safely
def run_workflow():
    try:
        # Check if an event loop is already running (e.g., in Jupyter)
        loop = asyncio.get_running_loop()
        # If loop exists, use it to run the coroutine
        return loop.create_task(main())
    except RuntimeError:
        # No running loop, safe to use asyncio.run
        return asyncio.run(main())

# Run the workflow
if __name__ == "__main__":
    run_workflow()

Waiter: Greet 🙏 the guest
Waiter: Provide Guest with water 🥤
Waiter: Provide guest the menu and confirm order
Waiter: Serve the guest 🍕
Waiter: Provide guest his/her bill
Guest: Clear his/her bill 💵
Waiter: Confirms the payment ✅
Finished txn:: Waiter: Confirms the payment ✅


In [29]:
##@ Drawing workflows:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(w)

workflow_all_flows.html


In [32]:
from IPython.display import display, HTML

with open('/kaggle/working/workflow_all_flows.html', 'r') as f:
    html_content = f.read()
display(HTML(html_content))

### State Management:

In [42]:
from llama_index.core.workflow import Event, Context
from llama_index.core.agent.workflow import ReActAgent


class ProcessingEvent(Event):
    intermediate_result: str


class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent, ctx: Context) -> ProcessingEvent:
        # Process initial data
        await ctx.set("query", "What is the capital of France?")
        return ProcessingEvent(intermediate_result="Step 1 complete")

    @step
    async def step_two(self, ev: ProcessingEvent, ctx: Context) -> StopEvent:
        # Use the intermediate result
        query = await ctx.get("query")
        print(f"Query: {query}")
        final_result = f"Finished processing: {ev.intermediate_result}"
        return StopEvent(result=final_result)


w = MultiStepWorkflow(timeout=10, verbose=False)
result = await w.run()
result

Query: What is the capital of France?


'Finished processing: Step 1 complete'

### MultiAgent Workflows:

In [44]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.agent.workflow import AgentWorkflow


def add(a: int, b: int)-> int:
    """Adds two ints"""
    return a+b

def mul(a: int, b: int)-> int:
    """Multiplies two ints"""
    return a*b


llm= HuggingFaceInferenceAPI(model_name = "Qwen/Qwen2.5-Coder-32B-Instruct")

#> We cant pass functions directly without the function tool

mul_agent= ReActAgent(
    name = "multiply_agent", 
    description = "Is able to multiply two ints", 
    system_prompt = "Assistant that can use a tool to multiply two numbers", 
    tools = [mul], 
    llm= llm
)

add_agent = ReActAgent(
    name = "add_agent", 
    description = "Is able to add two ints", 
    system_prompt = "Assistant that can use tool to add tewo numbers", 
    tools = [add], 
    llm = llm
)

workflow = AgentWorkflow(
    agents=[mul_agent, add_agent],
    root_agent="multiply_agent"
)

# Run the system
response = await workflow.run(user_msg="Can you add 56 and 13?")
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='69')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'to_agent': 'add_agent', 'reason': 'The user wants to add two numbers, and the add_agent is better suited for this task.'}, tool_id='83217015-2840-4bb3-8548-a327277c0654', tool_output=ToolOutput(content='Agent add_agent is now handling the request due to the following reason: The user wants to add two numbers, and the add_agent is better suited for this task..\nPlease continue with the current request.', tool_name='handoff', raw_input={'args': (), 'kwargs': {'to_agent': 'add_agent', 'reason': 'The user wants to add two numbers, and the add_agent is better suited for this task.'}}, raw_output='Agent add_agent is now handling the request due to the following reason: The user wants to add two numbers, and the add_agent is better suited for this task..\nPlease continue with the curre